In [2]:
import wandb

In [ ]:
%wandb login

wandb: Currently logged in as: shadaevf (shadaevf-rtu-mirea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Часть первая

In [ ]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None # function
        self._prev = set(_children) # set of Value objects
        self._op = _op # the op that produced this node, string ('+', '-', ....)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _op="+", _children=(self, other))

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _op="*", _children=(self, other))

        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data

        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, _op="**", _children=(self, ))

        def _backward():
            self.grad += out.grad * (other * ((self.data) ** (other-1)))

        out._backward = _backward

        return out

    def relu(self):
        out = Value(self.data if self.data > 0 else 0, _op="relu", _children=(self, ))

        def _backward():
            self.grad += out.grad * (1 if self.data > 0 else 0)
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [ ]:
import random

class Module:

    def zero_grad(self):
        for parameter in self.parameters():
          parameter.grad=0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin: int, nonlin=True):
        self.w = [Value(random.normalvariate(0, 2 / nin)) for _ in range(nin)]
        self.b = Value(random.normalvariate(0, 2 / nin))
        self.nonlin = nonlin

    def __call__(self, x):
        act = 0
        for i, el in enumerate(x):
          act += el * self.w[i]
        act += self.b
        return act.relu() if self.nonlin else act


    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = []
        for neuron in self.neurons:
          out.append(neuron(x))
        return out[0] if len(out) == 1 else out

    def parameters(self):
        out = []
        for neuron in self.neurons:
          out += neuron.parameters()
        return out

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

In [ ]:
def train(model, y, X, learning_rate=0.001, epochs=10, show=1):
    for k in range(epochs):

        # forward
        y_pred = []
        for row in X:
          y_pred.append(model(row))

        # calculate loss (mean square error)
        loss = 0
        for i in range(len(y_pred)):
          for j in range(len(y_pred[0])):
            loss += (y_pred[i][j] - y[i][j]) ** 2
        loss /= len(y_pred) * len(y_pred)

        # backward (zero_grad + backward)
        model.zero_grad()
        loss.backward()

        # update
        for p in model.parameters():
          p.data -= learning_rate * p.grad

        if k % show == 0:
            print(f"step {k} loss {loss.data}")

        wandb.log({"loss": loss.data})


In [ ]:
model = Layer(3, 2)

xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0]
]
ys = [[1.0, 1.0], [-1.0, 1.0], [-1.0, -1.0], [-1.0, 1.0]]

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="ml_pr_2",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="myautograd"
      )

train(model, ys, xs, learning_rate=0.3, epochs=30)

# Часть 2

In [ ]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ray.air.integrations.wandb import WandbLoggerCallback

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [4]:
# Загрузка данных
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5), (0.5))]
    )

    trainset = torchvision.datasets.EMNIST(
        root=data_dir, split='letters', train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.EMNIST(
        root=data_dir, split='letters', train=False, download=True, transform=transform
    )

    return trainset, testset

In [11]:
# Делаем нейронную сеть
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 27)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
def train_emnist(config, data_dir=None):
  # Создаём сеть
  net = NeuralNetwork()
  # Выбираем устройство
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda"
  net.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
  # Если уже была сохранена какая-то модель, то достаём её и продолжаем тюнить
  checkpoint = get_checkpoint()
  if checkpoint:
      with checkpoint.as_directory() as checkpoint_dir:
          data_path = Path(checkpoint_dir) / "data.pkl"
          with open(data_path, "rb") as fp:
              checkpoint_state = pickle.load(fp)
          start_epoch = checkpoint_state["epoch"]
          net.load_state_dict(checkpoint_state["net_state_dict"])
          optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
  else:
      start_epoch = 0

  trainset, testset = load_data(data_dir)

  val_size = int(len(trainset) * 0.2)
  train_subset, val_subset = random_split(
      trainset, [len(trainset) - val_size, val_size]
  )

  trainloader = torch.utils.data.DataLoader(
      train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
  )
  valloader = torch.utils.data.DataLoader(
      val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
  )

  for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "wb") as fp:
                pickle.dump(checkpoint_data, fp)

            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report(
                {"loss": val_loss / val_steps, "accuracy": correct / total},
                checkpoint=checkpoint,
            )

  print("Finished Training")

In [10]:
# Пишем тест
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [11]:
# Сам процесс тюнинга
def main(num_samples=10, max_num_epochs=10):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {

        "lr": tune.grid_search([0.1, 0.01, 0.001]),
        "batch_size": tune.grid_search([16, 32, 64]),
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
    results = tune.run(
        partial(train_emnist, data_dir=data_dir),
        resources_per_trial={"cpu":2, "gpu": 1},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        callbacks=[WandbLoggerCallback(project="ML2_2_Part_2")]

    )

    best_trial = results.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = NeuralNetwork()
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"

    best_trained_model.to(device)

    best_checkpoint = results.get_best_checkpoint(trial=best_trial, metric="accuracy", mode="max")
    with best_checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / "data.pkl"
        with open(data_path, "rb") as fp:
            best_checkpoint_data = pickle.load(fp)

        best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])
        test_acc = test_accuracy(best_trained_model, device)
        print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=1, max_num_epochs=10)

100%|██████████| 562M/562M [01:14<00:00, 7.51MB/s]
2025-03-26 14:34:25,213	INFO worker.py:1852 -- Started a local Ray instance.
2025-03-26 14:34:26,538	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-03-26 14:34:26,597	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2025-03-26 14:34:26,600	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2025-03-26 14:34:26,613	INFO wandb.py:318 -- Already logged into W&B.


+---------------------------------------------------------------------+
| Configuration for experiment     train_emnist_2025-03-26_14-34-26   |
+---------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator              |
| Scheduler                        AsyncHyperBandScheduler            |
| Number of trials                 9                                  |
+---------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_emnist_2025-03-26_14-34-26

Trial status: 9 PENDING
Current time: 2025-03-26 14:34:26. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------------------------------+
| Trial name                 status        lr     batch_size |
+------------------------------------------------------------+
| train_emnist_6b36b_00000   PENDING    0.1               16 |
| train_

(_WandbLoggingActor pid=5213) wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
(_WandbLoggingActor pid=5213) wandb: Currently logged in as: shadaevf (shadaevf-rtu-mirea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=5213) wandb: Tracking run with wandb version 0.19.8
(_WandbLoggingActor pid=5213) wandb: Run data is saved locally in /tmp/ray/session_2025-03-26_14-34-21_606425_267/artifacts/2025-03-26_14-34-26/train_emnist_2025-03-26_14-34-26/driver_artifacts/train_emnist_6b36b_00000_0_batch_size=16,lr=0.1000_2025-03-26_14-34-26/wandb/run-20250326_143435-6b36b_00000
(_WandbLoggingActor pid=5213) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=5213) wandb: Syncing run train_emnist_6b36b_00000
(_WandbLoggingActor pid=5213) wandb: ⭐️ View project at https://wandb.ai/shadaevf-rtu-mirea/ML2_2_Part_2
(_WandbLoggingActor pid=5213) wandb: 🚀 View run at https://wan

(func pid=5148) [1,  2000] loss: 3.267
(func pid=5148) [1,  4000] loss: 1.636

Trial status: 1 RUNNING | 8 PENDING
Current time: 2025-03-26 14:34:56. Total running time: 30s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------------------------------+
| Trial name                 status        lr     batch_size |
+------------------------------------------------------------+
| train_emnist_6b36b_00000   RUNNING    0.1               16 |
| train_emnist_6b36b_00001   PENDING    0.1               32 |
| train_emnist_6b36b_00002   PENDING    0.1               64 |
| train_emnist_6b36b_00003   PENDING    0.01              16 |
| train_emnist_6b36b_00004   PENDING    0.01              32 |
| train_emnist_6b36b_00005   PENDING    0.01              64 |
| train_emnist_6b36b_00006   PENDING    0.001             16 |
| train_emnist_6b36b_00007   PENDING    0.001             32 |
| train_emnist_6b36b_00008   PENDING    0.001           

(func pid=5148) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_emnist_2025-03-26_14-34-26/train_emnist_6b36b_00000_0_batch_size=16,lr=0.1000_2025-03-26_14-34-26/checkpoint_000000)
2025-03-26 14:35:14,503	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 1.219 s, which may be a performance bottleneck.
2025-03-26 14:35:14,505	WARNING util.py:201 -- The `process_trial_result` operation took 1.222 s, which may be a performance bottleneck.
2025-03-26 14:35:14,507	WARNING util.py:201 -- Processing trial results took 1.224 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2025-03-26 14:35:14,507	WARNING util.py:201 -- The `process_trial_result` operation took 1.224 s, which may be a performance bottleneck.


...


Trial status: 8 TERMINATED | 1 RUNNING
Current time: 2025-03-26 15:10:32. Total running time: 36min 5s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------------------------------------------------------------------------------------+
| Trial name                 status          lr     batch_size     iter     total time (s)       loss     accuracy |
+------------------------------------------------------------------------------------------------------------------+
| train_emnist_6b36b_00008   RUNNING      0.001             64                                                     |
| train_emnist_6b36b_00000   TERMINATED   0.1               16       10           378.584    3.27884     0.0369792 |
| train_emnist_6b36b_00001   TERMINATED   0.1               32        4           116.161    3.26794     0.0384215 |
| train_emnist_6b36b_00002   TERMINATED   0.1               64       10           246.891    2.80796     0.165064  |


(func pid=16642) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_emnist_2025-03-26_14-34-26/train_emnist_6b36b_00008_8_batch_size=64,lr=0.0010_2025-03-26_14-34-26/checkpoint_000000)
2025-03-26 15:10:45,439	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 1.798 s, which may be a performance bottleneck.
2025-03-26 15:10:45,442	WARNING util.py:201 -- The `process_trial_result` operation took 1.802 s, which may be a performance bottleneck.
2025-03-26 15:10:45,445	WARNING util.py:201 -- Processing trial results took 1.805 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2025-03-26 15:10:45,450	WARNING util.py:201 -- The `process_trial_result` operation took 1.810 s, which may be a performance bottleneck.
2025-03-26 15:10:45,511	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_emnist_2025-03-26_14-34-26' i


Trial train_emnist_6b36b_00008 finished iteration 1 at 2025-03-26 15:10:45. Total running time: 36min 18s
+--------------------------------------------------------------+
| Trial train_emnist_6b36b_00008 result                        |
+--------------------------------------------------------------+
| checkpoint_dir_name                        checkpoint_000000 |
| time_this_iter_s                                    26.67169 |
| time_total_s                                        26.67169 |
| training_iteration                                         1 |
| accuracy                                             0.65869 |
| loss                                      1.1793996095657349 |
+--------------------------------------------------------------+
Trial train_emnist_6b36b_00008 saved a checkpoint for iteration 1 at: (local)/root/ray_results/train_emnist_2025-03-26_14-34-26/train_emnist_6b36b_00008_8_batch_size=64,lr=0.0010_2025-03-26_14-34-26/checkpoint_000000

Trial train_emnist_6b36b_

(_WandbLoggingActor pid=16701) wandb: uploading history steps 0-0, summary
(_WandbLoggingActor pid=16701) wandb:                                                                                
(_WandbLoggingActor pid=16701) wandb: 
(_WandbLoggingActor pid=16701) wandb: Run history:
(_WandbLoggingActor pid=16701) wandb:                 accuracy ▁
(_WandbLoggingActor pid=16701) wandb: iterations_since_restore ▁
(_WandbLoggingActor pid=16701) wandb:                     loss ▁
(_WandbLoggingActor pid=16701) wandb:       time_since_restore ▁
(_WandbLoggingActor pid=16701) wandb:         time_this_iter_s ▁
(_WandbLoggingActor pid=16701) wandb:             time_total_s ▁
(_WandbLoggingActor pid=16701) wandb:                timestamp ▁
(_WandbLoggingActor pid=16701) wandb:       training_iteration ▁
(_WandbLoggingActor pid=16701) wandb: 
(_WandbLoggingActor pid=16701) wandb: Run summary:
(_WandbLoggingActor pid=16701) wandb:                 accuracy 0.65869
(_WandbLoggingActor pid=16701) wandb

Best trial test set accuracy: 0.8920673076923077


In [12]:
def train_normal(config, data_dir=None):
  # Создаём сеть
  net = NeuralNetwork()
  # Выбираем устройство
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda"
  net.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.RMSprop(net.parameters(), lr=config["lr"])

  trainset, testset = load_data(data_dir)

  trainloader = torch.utils.data.DataLoader(
      trainset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
  )
  testloader = torch.utils.data.DataLoader(
      testset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
  )

  for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0
            wandb.log({"train/loss": loss})

        # test loss
        test_loss = 0.0
        test_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(testloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                test_loss += loss.cpu().numpy()
                test_steps += 1

        wandb.log({"test/loss": test_loss / test_steps, "test/accuracy": correct / total})

  print("Finished Training")

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="ml2_2_part_3",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="RMSprop"
      )
data_dir = os.path.abspath("./data")
train_normal(config={"lr":0.001, "batch_size":16}, data_dir = data_dir)
wandb.finish()

[1,  2000] loss: 1.133
[1,  4000] loss: 0.354
[1,  6000] loss: 0.203
[2,  2000] loss: 0.522
[2,  4000] loss: 0.257
[2,  6000] loss: 0.171
[3,  2000] loss: 0.461
[3,  4000] loss: 0.238
[3,  6000] loss: 0.159
[4,  2000] loss: 0.442
[4,  4000] loss: 0.228
[4,  6000] loss: 0.156
[5,  2000] loss: 0.438
[5,  4000] loss: 0.221
[5,  6000] loss: 0.151
[6,  2000] loss: 0.420
[6,  4000] loss: 0.223
[6,  6000] loss: 0.148
[7,  2000] loss: 0.411
[7,  4000] loss: 0.217
[7,  6000] loss: 0.149
[8,  2000] loss: 0.416
[8,  4000] loss: 0.214
[8,  6000] loss: 0.146
[9,  2000] loss: 0.401
[9,  4000] loss: 0.216
[9,  6000] loss: 0.144
[10,  2000] loss: 0.408
[10,  4000] loss: 0.219
[10,  6000] loss: 0.145
Finished Training


test/accuracy,▁▁▅▄▇▅▅▇▆█
test/loss,▅▄▁▄▂▅▄▃█▄
train/loss,▆▅▂▄▄▂▂▄▁█▁▂▄▅▃▆▆▄▆▂▃▂▃▂▄▄▃▂▂▄▁▅▂▁▆▂▂▁▁▂
test/accuracy,0.86183
test/loss,0.53552
train/loss,0.38444
